**Connect google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
#clonining Github repo of Darknet for transfer learning
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
#compile Yolo darknet using gpu and opencv as both will be used 
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

In [ ]:
# get yolo config file for training(https://pjreddie.com/darknet/yolo/)
# based on Darknet author instruction for weights
# https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects
# our custom objet is floor in bulding here 
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg
#below part will automatically change the .cfg file during runtime, so saves you file from uploading
!sed -i 's/batch=1/batch=64/' cfg/yolov3.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3.cfg

In [ ]:
# directory to save weights of training and backing up weights automatically in case of crash
!mkdir "/mydrive/floor_counter"
# based on Darknet author instruction for weights
# https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects
!echo "floor" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/floor_counter' > data/obj.data
!mkdir data/obj
# download the weights for the convolutional layers here (76 MB) 
#instruction from author https://pjreddie.com/darknet/yolo/
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
#extract training image zip file in above mounted folder "floor_counter" in google drive 
#this zip contains images and labbled value from labelimg tool, which contains additional .text file apart from image; having cordinate of 
#floor labelling in form of square(so squares coordinate)
!unzip /mydrive/floor_counter/training_images.zip -d data/obj

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import os
import re
import glob

#image_segment_cordinate = glob.glob(r"data/obj/*.txt")
#image_segment_cordinate= C:/user/xx/project/darknet/build/darknet/x64/data/obj.names
# while running in local machine use above code 
image_segment_cordinate = glob.glob(r"data/obj/*.txt")

for num, txt_file_path in enumerate(image_segment_cordinate):
# image size and getting coordinate of segmetaion 
    with open(txt_file_path, "r") as line_value:
        lines = line_value.readlines()
        text_converted = []
        for line in lines:
            numbers = re.findall("[0-9.]+", line)
            if numbers:
                # cordinates of floor in images
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
        # Write file
        with open(txt_file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)
#joining image and segmentaion cordinate file to get complete training file
#this will be fed to Draknet Network for CNN ops
images_list = glob.glob("data/obj/*.jpg")
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close()      # this is training text file which is closed here           

In [ ]:
# running Darknet with out prepared training data file
#this training can go on very long time
# in google drive folder it will keep on making backup of trained weights
#suggested no of iteration is minimum 1000 but anytime it can be stopped and last weight obatined in g-drive can be used for inference
#log - creates lod file for us
!./darknet detector train data/obj.data cfg/yolov3.cfg darknet53.conv.74 -dont_show >> /mydrive/floor_counter/.log

In [ ]:
!python3 plotTrainLoss.py /mydrive/floor_counter/.log